
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [2]:
batch_size = 10
epochs = 10

In [3]:
linear_layer_2d = nn.Linear(in_features=68, out_features=128)
flatten = nn.Flatten()
input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
print(input_2d.shape)
output_2d = linear_layer_2d(input_2d)
print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)



torch.Size([2, 2, 68])
output_2d tensor([[[-5.4272e-01,  4.4423e-01,  6.0596e-01, -2.6507e-02, -1.0593e+00,
           3.2728e-01, -7.2961e-01, -6.5087e-02,  1.1298e+00,  4.8103e-01,
           6.8498e-01,  7.2168e-01, -7.0304e-01, -1.0294e+00,  1.3081e+00,
          -4.7290e-01,  3.1143e-01,  3.4690e-01, -6.1350e-01, -6.2106e-01,
           1.3366e-02,  1.2418e+00,  3.2735e-01,  1.1143e-01,  3.9702e-01,
           8.4877e-01, -2.9535e-01, -3.7765e-01, -8.7391e-01,  4.2160e-01,
          -2.5399e-01, -6.9817e-01, -1.2811e-01, -6.2158e-01,  3.7030e-01,
           4.7983e-01, -6.6515e-02, -3.2970e-01, -5.8664e-01, -4.1730e-01,
           2.6763e-02, -1.1138e-01, -4.4584e-02, -1.5761e-01,  5.1167e-02,
          -9.0404e-01, -3.3337e-01,  2.9068e-01, -1.9146e+00, -7.7935e-01,
           8.8068e-02,  4.2703e-01, -6.9054e-01, -3.2494e-01,  1.9030e-01,
           1.1646e+00, -1.0978e-01, -1.7878e-01,  8.4153e-01,  6.1945e-01,
           2.3311e-01, -8.4344e-01,  8.4707e-01,  4.3089e-01,  6.79

In [4]:
'''
testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)
'''

'\ntestload = pd.read_csv("./outimg/Train/facefeature.csv")\nlandmarks = np.array(testload.iloc[0, 1:])\nlandmarks = landmarks.astype(\'float\').reshape(-1, 2)\nprint(landmarks)\n\ndataiter = iter(testload)\nlandmark = next(dataiter)\nprint(landmark)\n# print(landmarks.shape)\n'

In [5]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([10, 136])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
X type torch.float32
y type torch.float32


In [6]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [7]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, num_classes),
            #nn.ReLU(),
            #nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=33, bias=True)
  )
)


In [8]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        # print(pred)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
# Training
epochs = 1000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.312433  [    0/  100]
Epoch 2
-------------------------------
loss: 0.279359  [    0/  100]
Epoch 3
-------------------------------
loss: 0.305074  [    0/  100]
Epoch 4
-------------------------------
loss: 0.281749  [    0/  100]
Epoch 5
-------------------------------
loss: 0.293791  [    0/  100]
Epoch 6
-------------------------------
loss: 0.291111  [    0/  100]
Epoch 7
-------------------------------
loss: 0.282101  [    0/  100]
Epoch 8
-------------------------------
loss: 0.288735  [    0/  100]
Epoch 9
-------------------------------
loss: 0.275069  [    0/  100]
Epoch 10
-------------------------------
loss: 0.288075  [    0/  100]
Epoch 11
-------------------------------
loss: 0.290006  [    0/  100]
Epoch 12
-------------------------------
loss: 0.287282  [    0/  100]
Epoch 13
-------------------------------
loss: 0.277202  [    0/  100]
Epoch 14
-------------------------------
loss: 0.283883  [    0/  100]
Epoch 15
------

In [11]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [13]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4853, 0.4713, 0.4870, 0.5052, 0.5067, 0.5121, 0.4746, 0.4844, 0.5185,
         0.4949, 0.5073, 0.4779, 0.4920, 0.4993, 0.4753, 0.4864, 0.5069, 0.5249,
         0.4912, 0.5100, 0.4936, 0.4999, 0.4842, 0.4933, 0.5186, 0.4802, 0.5037,
         0.4860, 0.4739, 0.4987, 0.5113, 0.5123, 0.4727],
        [0.4793, 0.4726, 0.4820, 0.5064, 0.5086, 0.5071, 0.4718, 0.5013, 0.5213,
         0.5034, 0.5010, 0.4987, 0.4888, 0.5025, 0.4796, 0.4700, 0.5116, 0.5312,
         0.4936, 0.5148, 0.4849, 0.4957, 0.4912, 0.4909, 0.5065, 0.4845, 0.5220,
         0.4845, 0.4815, 0.5008, 0.5141, 0.5102, 0.4780],
        [0.4764, 0.4658, 0.4747, 0.5013, 0.5085, 0.5033, 0.4702, 0.4924, 0.5126,
         0.4971, 0.5031, 0.4839, 0.4809, 0.4994, 0.4748, 0.4724, 0.5042, 0.5242,
         0.4894, 0.5040, 0.4816, 0.5008, 0.4897, 0.4895, 0.5031, 0.4774, 0.5104,
         0.4798, 0.4701, 0.4914, 0.5139, 0.5031, 0.4727],
        [0.4648, 0.4613, 0.4686, 0.4855, 0.5013, 0.4953, 0.4689, 0.4874, 0.5